## 🎓 Persona : Léa, jeune investisseuse étudiante

**Profil :**
- 👩 24 ans, diplômée de l'EM Lyon
- 💼 Première expérience professionnelle après 2 ans d'alternance
- 💰 Aide parentale pour le financement + épargne personnelle (~15 000 €)
- 🎯 Objectif : réaliser un **premier investissement locatif** dans une **ville étudiante dynamique**

---

### 💡 Objectif d'investissement
> Trouver le **meilleur investissement locatif étudiant** possible avec un **budget global de 200 000 €**,  
> en ciblant un **studio à Lille**, tout en comparant brièvement avec un **T1 à Angers ou Nancy**.

---

### 💰 Hypothèses financières
| Élément | Montant estimé |
|----------|----------------|
| Prix d'achat visé | 160 000 – 180 000 € |
| Apport personnel | 15 000 € |
| Prêt immobilier estimé | 180 000 € sur 20 ans |
| Budget total (frais inclus) | **≈ 200 000 €** |
| Objectif de rentabilité brute | **≥ 5 %** |

---

### 🏙️ Cibles principales
| Ville | Type de bien | Prix moyen au m² | Loyer moyen mensuel | Observations |
|-------|---------------|------------------|---------------------|---------------|
| **Lille** | Studio (20–25 m²) | ~4 500 €/m² | 550–600 € | Marché étudiant tendu, forte demande locative |
| **Angers** | T1 (25–30 m²) | ~3 200 €/m² | 450–500 € | Ville très dynamique, bonne rentabilité brute |
| **Nancy** | T1 (25–30 m²) | ~2 800 €/m² | 420–470 € | Marché abordable, bon rapport prix/rentabilité |

---

### 🔍 Besoins data de Léa
- Identifier **les quartiers les plus rentables** à Lille (ou dans des villes comparables)
- Comparer avec **la rentabilité moyenne en France**
- Analyser l'**évolution du prix au m² et des loyers étudiants** depuis 5 ans
- Calculer la **rentabilité locative brute et nette** par quartier
- Visualiser les **zones à forte concentration étudiante**
- Fournir une **recommandation finale : "où investir avec 200k€ ?"**
- Évaluer le **taux de vacance locative** par quartier pour anticiper les périodes creuses (notamment l'été où les étudiants quittent les logements)
- Analyser la **proximité des transports en commun** et des universités/grandes écoles pour identifier les zones les plus attractives pour les étudiants
- Estimer les **charges de copropriété moyennes** par type de bien et par quartier pour affiner le calcul de rentabilité nette
- Identifier les **opportunités de biens nécessitant des travaux** (décote à l'achat) pour maximiser la plus-value à long terme

---

### 🧭 Objectif du notebook
Créer un outil interactif permettant à Léa de :
1. Comparer la rentabilité d'un **studio à Lille** avec celle d'un **T1 à Angers ou Nancy**  
2. Explorer visuellement les **zones à potentiel locatif élevé**  
3. Obtenir une **recommandation automatique** en fonction de son budget et de ses préférences

## Import des bibliothèques ##

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import requests, zipfile
from io import BytesIO
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import plotly.express as px

### 1e vision : Analyse générale en France ###

Etude de la rentabilité moyenne en France

In [ ]:
print("Partie Valentine")

Evolution du prix au m² et des loyers étudiants depuis 5 ans en France

In [ ]:
# --- Étape 1 : Télécharger et extraire le dossier temporairement ---
url_zip = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main/data_loyer.zip"
folder = "data_loyer"

response = requests.get(url_zip)
if response.status_code == 200:
    with zipfile.ZipFile(BytesIO(response.content)) as zip_ref:
        zip_ref.extractall(folder)
else:
    raise Exception(f"❌ Erreur lors du téléchargement (code {response.status_code})")

# --- Vérification du dossier ---
if not os.path.exists(folder):
    raise FileNotFoundError(f"❌ Dossier introuvable : {folder}")


# --- Liste pour stocker les données de chaque ville ---
all_data = []

# --- Lecture de tous les fichiers CSV ---
for file in os.listdir(folder):
    if file.endswith(".csv"):
        city_name = file.split("_")[-1].replace(".csv", "")  # extrait le nom de la ville
        df = pd.read_csv(os.path.join(folder, file), sep=";", encoding="ISO-8859-1")

        # Calcul du loyer moyen mensuel
        if "moyenne_loyer_mensuel" in df.columns:
            # Remplacement de la virgule par un point pour éviter les erreurs de conversion
            df["moyenne_loyer_mensuel"] = df["moyenne_loyer_mensuel"].astype(str).str.replace(",", ".")
            df["moyenne_loyer_mensuel"] = pd.to_numeric(df["moyenne_loyer_mensuel"], errors="coerce")
            mean_rent = df["moyenne_loyer_mensuel"].dropna().mean()
            all_data.append({"Ville": city_name, "Loyer_moyen": mean_rent})

# --- Création du DataFrame final ---
df_villes = pd.DataFrame(all_data)

# --- Tri par ordre alphabétique des villes ---
df_villes = df_villes.sort_values(by="Ville")

# --- Visualisation ---
plt.figure(figsize=(10,6))
plt.bar(df_villes["Ville"], df_villes["Loyer_moyen"], color='cornflowerblue')
plt.title("Loyer mensuel moyen par ville (2024)")
plt.ylabel("Loyer moyen (€ / mois)")
plt.xticks(rotation=45, ha="right")
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()


Visualiser les zones à forte concentration étudiante en France

In [ ]:
url_df_pop_communales = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main/POPULATION_MUNICIPALE_COMMUNES_FRANCE.xlsx"

df_populations_communales = pd.read_excel(url_df_pop_communales)

df_populations_communales.head()

In [ ]:
df_populations_communales.drop(['p13_pop', 'p14_pop', 'p15_pop', 'p16_pop', 'p17_pop', 'p18_pop', 'p19_pop', 'p20_pop'], axis=1, inplace=True)

# Garde les lignes où la colonne 'dep' contient le mot "Arrondissement"
df_filtered = df_populations_communales[df_populations_communales['libgeo'].str.contains("Arrondissement")]

df_filtered

In [ ]:
df = df_populations_communales.copy()

df['libgeo'] = df['libgeo'].replace(
    to_replace = [r'Paris.*', r'Lyon.*', r'Marseille.*'], 
    value = ['Paris', 'Lyon', 'Marseille'], 
    regex=True
)

treated_df = df.groupby('libgeo', as_index=False).agg({
    'objectid': 'first',   # garde le premier ID (ou autre logique)
    'reg': 'first',        # garde la première région
    'dep': 'first',        # idem pour département
    'cv': 'first', 
    'codgeo': 'first',
    'p21_pop': 'sum'
})

treated_df

In [ ]:
df_pop_100k = treated_df[treated_df['p21_pop'] > 100000]

df_pop_100k.sort_values(by='p21_pop', ascending=False)

In [ ]:
url_df_enseignement_sup = "https://huggingface.co/datasets/analysedonneesfoncieresdata/analyse_fonciere_data/resolve/main/fr-esr-atlas_regional-effectifs-d-etudiants-inscrits-detail_etablissements.csv"

df_enseignement_sup = pd.read_csv(url_df_enseignement_sup, delimiter=';')

df_enseignement_sup.head()

In [ ]:
# 1️⃣ Exclure les lignes avec "Étranger"
df_enseignement_sup = df_enseignement_sup[~df_enseignement_sup['département'].str.contains("Étranger")]

# 2️⃣ Garder uniquement les 2 premiers chiffres
df_enseignement_sup['dep'] = df_enseignement_sup['département'].str[:2]

# 3️⃣ Optionnel : supprimer les espaces et convertir en int
df_enseignement_sup['dep'] = df_enseignement_sup['dep'].str.strip()

df_enseignement_sup.head()

In [ ]:
df_enseignement_sup_treated = df_enseignement_sup.groupby('dep', as_index=False)[[
    'nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE',
    'dont femmes',
    'dont hommes'
]].sum()

df_enseignement_sup_treated.head()

In [ ]:
df_joined = pd.merge(df_pop_100k, df_enseignement_sup_treated, on='dep', how='inner')

df_joined.head()

In [ ]:
df_joined['Densité étudiante'] = df_joined['nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE'] / df_joined['p21_pop']

df_joined.head()

In [ ]:
# Initialiser le géocodeur
geolocator = Nominatim(user_agent="adam_geocoder_ece_paris")

# Limiteur de requêtes pour éviter les erreurs 429
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

def get_coordinates(ville):
    try:
        if pd.isna(ville):
            return pd.Series([None, None])
        location = geocode(f"{ville}, France")  # ajoute "France" pour plus de précision
        if location:
            return pd.Series([location.latitude, location.longitude])
        else:
            return pd.Series([None, None])
    except Exception as e:
        print(f"Erreur pour la ville '{ville}': {e}")
        return pd.Series([None, None])

# Appliquer à la colonne des villes
df_joined[["latitude", "longitude"]] = df_joined["libgeo"].apply(get_coordinates)

In [ ]:
df_top_students = df_joined.sort_values(
    by='nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE',
    ascending=False
).head(20)

df_top_density = df_joined.sort_values(
    by='Densité étudiante',
    ascending=False
).head(20)

In [ ]:
plt.figure(figsize=(10, 6)) # Taille du graphique 
plt.bar(df_top_students['libgeo'], df_top_students['nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE']) 
plt.title('Top 20 villes par nombre d’étudiants (population > 100k)') 
plt.xlabel('Villes') 
plt.xticks(rotation=45, ha='right', rotation_mode='anchor') 
plt.ylabel('Nombre d’étudiants (en million)') 
plt.grid(False) 

plt.figure(figsize=(10, 6)) # Taille du graphique 
plt.bar(df_top_density['libgeo'], df_top_density['Densité étudiante']) 
plt.title('Histogramme avec courbe de densité pour la colonne X') 
plt.xlabel('Villes') 
plt.xticks(rotation=45, ha='right', rotation_mode='anchor') 
plt.ylabel('Densité Etudiante') 
plt.grid(False) 
plt.show()

In [ ]:
# Carte interactive avec Plotly
fig = px.scatter_map(
    df_joined,
    lat="latitude",
    lon="longitude",
    hover_name="libgeo",
    hover_data=["nombre total d’étudiants inscrits hors doubles inscriptions université/CPGE", "Densité étudiante"],
    size="Densité étudiante",
    zoom=5,
    height=600
)

# Style cartographique (nécessite une Mapbox style)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(title="Répartition géographique des étudiants en France")

fig.show()

Évaluer le taux de vacance locative en France, et les zones propices à un taux élevé

In [ ]:
print("Partie Lucien")